# Fama-French 3-Factor Analysis
Goal: Regress a stock's returns on the Fama-French 3 factors to estimate factor loadings.


In [28]:
!pip install yfinance pandas-datareader --quiet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import yfinance as yf
from pandas_datareader import data as pdr

plt.style.use('default')


In [29]:
# Real data example: AAPL (uncomment to fetch)
# yf.pdr_override()
# prices = pdr.get_data_yahoo(['AAPL'], start='2018-01-01', end='2024-01-01')['Adj Close']
# stock_prices = prices['AAPL']

# Synthetic fallback:
np.random.seed(2)
dates = pd.date_range('2020-01-01', periods=500)
market = np.cumsum(np.random.normal(0,1,500)) + 100
stock = market * 1.1 + np.random.normal(0,5,500) + 5
stock_prices = pd.Series(stock, index=dates, name='Stock')
market_series = pd.Series(market, index=dates, name='Market')


In [30]:
# Online: you would download Fama-French factors from Kenneth French's data library.
# In Colab, you can fetch via pandas-datareader or manually upload CSV.

# For offline run we'll synthesize simple factor series (for demonstration):
np.random.seed(3)
factors = pd.DataFrame({
    'Mkt-RF': np.random.normal(0, 0.01, len(dates)),
    'SMB': np.random.normal(0, 0.005, len(dates)),
    'HML': np.random.normal(0, 0.005, len(dates)),
    'RF': np.zeros(len(dates))
}, index=dates)
factors.head()


,Mkt-RF,SMB,HML,RF
2020-01-01,0.017886,0.009077,0.000173,0.0
2020-01-02,0.004365,-0.004081,0.005660,0.0
2020-01-03,0.000965,0.001430,-0.012784,0.0
2020-01-04,-0.018635,-0.001993,-0.001345,0.0
2020-01-05,-0.002774,0.002181,-0.009548,0.0


In [31]:
# Compute returns
stock_ret = stock_prices.pct_change().dropna()
# Align to factors (drop first row if mismatch)
factors = factors.loc[stock_ret.index]

excess_ret = stock_ret - factors['RF']
X = factors[['Mkt-RF','SMB','HML']]
X = sm.add_constant(X)

model = sm.OLS(excess_ret, X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.4746
Date:                Fri, 12 Dec 2025   Prob (F-statistic):              0.700
Time:                        18:55:28   Log-Likelihood:                 646.32
No. Observations:                 499   AIC:                            -1285.
Df Residuals:                     495   BIC:                            -1268.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0023      0.003      0.758      0.4

### Interpretation & Extensions
- Coefficients show exposure to market, size (SMB) and value (HML).
- Extensions: use true Fama-French data (download CSV from Kenneth French), perform rolling regressions, test factor significance.
